# 03 Gemma

## Notebook setup

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:

# Define data path
INPUT_PATH = '/content/drive/MyDrive/266_Final_Project/Input_Data/NumHG-main/Dataset/'
PROCESSED_PATH = '/content/drive/MyDrive/266_Final_Project/Processed_Data/'

In [3]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
!pip install -q math-verify
!pip install -q word2number
#!pip install -U bitsandbytes
#!pip uninstall torch torchvision torchaudio
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

In [19]:
# Import libraries
import numpy as np
import pandas as pd
import torch

import transformers
import evaluate

import matplotlib.pyplot as plt
from datasets import Dataset, load_from_disk

from transformers import AutoTokenizer, ModernBertForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
from transformers import pipeline

import torch

import csv
from math_verify import parse, verify
from word2number import w2n
import pickle
import regex as re

In [5]:
# Set seed
np.random.seed(42)
pd_random_state = 24

## Intake of train/test data

In [6]:
# Read in processed HF data
train_hf = load_from_disk(PROCESSED_PATH + 'train_gemma3.hf')
test_hf = load_from_disk(PROCESSED_PATH + 'test_gemma3.hf')

In [7]:
train_hf[0]

{'context': "In what's believed to be the largest Hollywood divorce settlement in history, Mel Gibson is having to cough up $425 million—half of his estimated fortune—to ex-wife Robyn Denise Moore. The two, married 31 years and parents to seven children, didn't have a prenuptial agreement, so Moore is entitled to half his wealth under California law. Moore, a former dental nurse, is also entitled to half of any future residuals from films Gibson, 55, worked on while the two were married, reports People. The settlement was finalized by a judge last week. Gibson earned $600 million alone from his The Passion of the Christ. He also made money from real estate, including a Fiji island he bought for $15 million. Moore, 55, filed for divorce from Gibson in 2009 after he began a highly publicized, crazed affair with Oksana Grigorieva. Moore and Gibson met in the late 1970s when they were both tenants in a house in Australia. Of all their children, only a 12-year-old son is a minor and subject

## Fine-tuning on math problems dataset

## Prompt output

In [9]:
model_id = "google/gemma-3-1b-it"

In [12]:
pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)

model.safetensors:  52%|#####1    | 1.04G/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda


In [13]:
def predict_masked_token_gemma(context, question):
  input_string = "Context: {0} Question: Based on the context, what number should replace the [MASK] token in the given headline. Only provide the number as response. Headline: {1}".format(context,question)
  messages = [
      [
          {
              "role": "system",
              "content": [{"type": "text", "text": "You are a fill-in-the-blank expert. You read the entire context and give the most accurate response. Make sure to account for K (thousands) and M (millions)."},]
          },
          {
              "role": "user",
              "content": [{"type": "text", "text": input_string},]
          },
      ],
    ]
  output = pipe(messages, max_new_tokens=5)
  return output[0][0]['generated_text'][2]['content']


def process_data(data):
  #data["predicted_labels"] = predict_masked_token_gemma(data["context"],data["question"])
  #return data
  return predict_masked_token_gemma(data["context"],data["question"])

In [ ]:
test_hf = test_hf.map(process_data)
## RUNS OUT OF RAM

In [20]:
predictions = []
for i in range(0,2774): # range(0,test_hf)
  predictions.append(process_data(test_hf[i]))

In [10]:
file_path_pickle = PROCESSED_PATH + "gemma_test_predictions.pkl"

In [21]:
## To save predictions as pickle file
#with open(file_path_pickle, "wb") as file:
#    pickle.dump(predictions, file)

In [11]:
with open(file_path_pickle, 'rb') as file:
    predictions = pickle.load(file)

In [54]:
def convert_word_to_number(word):
  #Converts words to numbers (as string)
  #print(word)
  try:
    word = re.findall(r'\d+\.?\d*', word)[0]
  except IndexError:
    pass

  try:
    number = str(w2n.word_to_num(word))
    return number
  except ValueError:
        #print(f"Error: '{word}' could not be converted to a number.")
        return word


In [69]:
# Calculate accuracy
matches = []
for i in range(len(predictions)):
  if verify(test_hf['labels'][i],predictions_nums[i]):
        #test_hf['labels'][i] == predictions[i]:       # equals operation is faster than using math_verify
        #verify(test_hf['labels'][i],predictions[i]):
    matches.append(1)
  else:
    matches.append(0)

accuracy = sum(matches)/len(matches)
print(accuracy)

0.3439077144917087


## TESTING

In [ ]:
# Batch processing example
def process_batch(batch):
    prompts = [f"Q: {q}\nC: {c}\nA:"
               for q,c in zip(batch['question'], batch['context'])]
    outputs = qa_pipe(prompts, batch_size=8)
    return {"predictions": [out[0]['generated_text'] for out in outputs]}

dataset = dataset.map(process_batch, batched=True)

In [ ]:
def predict_masked_token_gemma_batched(batch):
  messages = []
  for context,question in zip(batch['context'],batch['question']):
    input_string = "Context: {0} Question: Based on the context, what number should replace the [MASK] token in the given headline. Only provide the number as response. Headline: {1}".format(context,question)
    messages.append([
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a fill-in-the-blank expert. You read the entire context and give the most accurate response. Make sure to account for K (thousands) and M (millions)."},]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": input_string},]
            },
        ],
    )
  outputs = pipe(messages, max_new_tokens=5, batch_size=2)
  return {"predictions": [out[0]['generated_text'][2]['content'] for out in outputs]}


#def process_data(data):
#  data["predicted_labels"] = predict_masked_token_gemma(data["context"],data["question"])
#  return data

In [ ]:
subset.select(range(1))['context']

["In what's believed to be the largest Hollywood divorce settlement in history, Mel Gibson is having to cough up $425 million—half of his estimated fortune—to ex-wife Robyn Denise Moore. The two, married 31 years and parents to seven children, didn't have a prenuptial agreement, so Moore is entitled to half his wealth under California law. Moore, a former dental nurse, is also entitled to half of any future residuals from films Gibson, 55, worked on while the two were married, reports People. The settlement was finalized by a judge last week. Gibson earned $600 million alone from his The Passion of the Christ. He also made money from real estate, including a Fiji island he bought for $15 million. Moore, 55, filed for divorce from Gibson in 2009 after he began a highly publicized, crazed affair with Oksana Grigorieva. Moore and Gibson met in the late 1970s when they were both tenants in a house in Australia. Of all their children, only a 12-year-old son is a minor and subject to a custo

In [ ]:
subset = train_hf.select(range(2))

subset = subset.map(predict_masked_token_gemma_batched, batched=True)

In [ ]:
subset[0]

In [ ]:

messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a fill-in-the-blank expert. You read the entire context and give "},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Context: Ukraine PM Arseniy Yatsenyuk will head to Washington this week for talks on the Crimea situation, Fox News reports. Yatsenyuk announced his US visit, which he said will seek a  resolution of the situation in Ukraine,  after a government meeting. Meanwhile, Yatsenyuk was talking tough today, adds the AP, vowing at a Kiev celebration of the country's great nationalist poet that  This is our land. Our fathers and grandfathers have spilled their blood for this land. And we won't budge a single centimeter from Ukrainian land. Let Russia and its president know this.  Crimea was throwing a party of its own, with 4,000-plus people gathering in the capital of Simferopol to throw their support behind unification with Russia.  Russians are our brothers,  Crimea's parliament speaker told the crowd.  We are going back home to the motherland.  His speech was greeted with cries of  Russia! Russia!,  notes the AP. A smaller gathering of 500 or so turned out to voice opposition to the referendum.  We will not allow a foreign boot that wants to stand on the heads of our children,  said one protester. Question: Based on the context, what number should replace the [MASK] token in the given headline. Only provide the number as response. Headline: Ukraine PM: We Won't Budge ' [MASK] Centimeter'."},]
            #"content": [{"type": "text", "text": "Based on the context, what word should replace the [MASK] token in the given headline. Only give one word as the answer. Headline: The sky is [MASK] in color."},]
        },
    ],
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a fill-in-the-blank expert. You read the entire context and give "},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Context:  Wonder Woman has soared to a $100.5 million box-office debut in North American theaters, making the Patty Jenkins-directed superhero film the biggest opening ever for a movie directed by a woman, the AP reports. According to studio estimates Sunday, the well-reviewed Wonder Woman surpassed industry expectations with one of the summer's biggest debuts. Starring Gal Gadot as the Amazonian warrior, Wonder Woman is the rare female-led film in an overwhelmingly male superhero landscape. It proved a hit with moviegoers, earning an A CinemaScore and, while skewing somewhat female, drew a fairly evenly split audience. Warner Bros. said 52% of the audience was female and 48% male. Last week's top film, Pirates of the Caribbean: Dead Men Tell No Tales, is expected to slide steeply for a distant second place. Question: Based on the context, what number should replace the [MASK] token in the given headline. Only provide the number as response. Headline: Wonder Woman Sets Record With $ [MASK] M Debut."},]
            #"content": [{"type": "text", "text": "Based on the context, what word should replace the [MASK] token in the given headline. Only give one word as the answer. Headline: The sky is [MASK] in color."},]
        },
    ],
]

outputs = pipe(messages, batch_size=1, max_new_tokens=5)


NameError: name 'pipe' is not defined

In [ ]:
outputs

NameError: name 'outputs' is not defined

In [11]:

messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a fill-in-the-blank expert. You read the entire context and give "},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Context:  Wonder Woman has soared to a $100.5 million box-office debut in North American theaters, making the Patty Jenkins-directed superhero film the biggest opening ever for a movie directed by a woman, the AP reports. According to studio estimates Sunday, the well-reviewed Wonder Woman surpassed industry expectations with one of the summer's biggest debuts. Starring Gal Gadot as the Amazonian warrior, Wonder Woman is the rare female-led film in an overwhelmingly male superhero landscape. It proved a hit with moviegoers, earning an A CinemaScore and, while skewing somewhat female, drew a fairly evenly split audience. Warner Bros. said 52% of the audience was female and 48% male. Last week's top film, Pirates of the Caribbean: Dead Men Tell No Tales, is expected to slide steeply for a distant second place. Question: Based on the context, what number should replace the [MASK] token in the given headline. Only provide the number as response. Headline: Wonder Woman Sets Record With $ [MASK] M Debut."},]
            #"content": [{"type": "text", "text": "Based on the context, what word should replace the [MASK] token in the given headline. Only give one word as the answer. Headline: The sky is [MASK] in color."},]
        },
    ],
]

output = pipe(messages, max_new_tokens=50)

NameError: name 'pipe' is not defined

In [ ]:
output[0][0]

{'generated_text': [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a fill-in-the-blank expert. You read the entire context and give '}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': "Context: Ukraine PM Arseniy Yatsenyuk will head to Washington this week for talks on the Crimea situation, Fox News reports. Yatsenyuk announced his US visit, which he said will seek a  resolution of the situation in Ukraine,  after a government meeting. Meanwhile, Yatsenyuk was talking tough today, adds the AP, vowing at a Kiev celebration of the country's great nationalist poet that  This is our land. Our fathers and grandfathers have spilled their blood for this land. And we won't budge a single centimeter from Ukrainian land. Let Russia and its president know this.  Crimea was throwing a party of its own, with 4,000-plus people gathering in the capital of Simferopol to throw their support behind unification with Russia.  Russians are our brothers,  Crimea's

In [ ]:
output[0][0]['generated_text'][2][]

'1'

In [ ]:
outputs = []
for i in range(1):
  input_string = "Context: {0} Question: Based on the context, what number should replace the [MASK] token in the given headline. Only provide the number as response. Headline: {1}".format(train_hf[i]['context'],train_hf[i]['question'])
  messages = [
      [
          {
              "role": "system",
              "content": [{"type": "text", "text": "You are a fill-in-the-blank expert. You read the entire context and give the most accurate response. Make sure to account for K (thousands) and M (millions)."},]
          },
          {
              "role": "user",
              "content": [{"type": "text", "text": input_string},]
          },
      ],
    ]
  output = pipe(messages, max_new_tokens=5)
  outputs.append(output[0][0]['generated_text'][2]['content'])

In [ ]:
outputs

['425']

In [67]:
float(predictions_nums[19])

1.0

In [68]:
float(test_hf['labels'][19])

1.0

In [65]:
parse(predictions_nums[19])

[1.00000000000000, '1.0']

In [66]:
parse(test_hf['labels'][19])

[1, '1']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
        },
    ],
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device).to(torch.bfloat16)


with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64)

outputs = tokenizer.batch_decode(outputs)

NameError: name 'model' is not defined

In [ ]:
# Import Gemma tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map="auto",
)

# Define prompt
prompt_head = "The following text contains a news article headline summary with a '[MASK]' token followed by the corresponding news article. Use the article to fill in the '[MASK]' token with the relevant number. "